In [ ]:
'''
# GPU BEGONE!
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu'
os.environ['PATH'] = '/home/fandav/.local/bin:/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/opt/anaconda3/envs/jupyter/bin:/opt/anaconda3/bin/:/snap/bin'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ
'''

In [2]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np
import pandas as pd

# To find out which devices your operations and tensors are assigned to
tf.debugging.set_log_device_placement(True)

In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [4]:
'''
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
'''

'\nfrom tensorflow.compat.v1 import ConfigProto\nfrom tensorflow.compat.v1 import InteractiveSession\n\nconfig = ConfigProto()\nconfig.gpu_options.allow_growth = True\nsession = InteractiveSession(config=config)\n'

In [5]:
physical_cpus = tf.config.list_physical_devices('CPU') 
logical_cpus = tf.config.list_logical_devices('GPU')
print(len(physical_cpus), "Physical CPUs,", len(logical_cpus), "Logical CPUs")

1 Physical CPUs, 1 Logical CPUs


In [6]:
'''
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
'''

'\ngpus = tf.config.list_physical_devices(\'GPU\')\nif gpus:\n    try:\n        for gpu in gpus:\n            tf.config.experimental.set_memory_growth(gpu, True)\n            logical_gpus = tf.config.list_logical_devices(\'GPU\')\n            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")\n    except RuntimeError as e:\n        print(e)\n'

In [ ]:
# tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased');

# model
model = TFBertModel.from_pretrained('bert-base-uncased');
model.summary()

In [ ]:
dataset = pd.read_csv('dataset.csv', index_col=0)
#display(dataset.loc['Content'].values)

In [ ]:
# grab content
dataset_content = dataset.loc['Content'].fillna('').values
#print(dataset_content)

# process content through BERT
encoded_strings = [tokenizer.encode(content, max_length=512, pad_to_max_length=True) for content in dataset_content]
encoded_vectors = tf.constant(encoded_strings)
outputs = model(encoded_vectors)

# obtain per line embeddings from all the artibles from BERT
last_hidden_states = outputs[0]

# obtain article embeddings from BERT
article_embeddings = outputs[1]

print(np.shape(outputs))
print(np.shape(outputs[0]))
print(np.shape(outputs[1]))

In [ ]:
_, num_articles = np.shape(outputs)
article_names = [f'Article {idx}' for idx in range(num_articles)]

# Look at some dimensionality reduction plots

In [ ]:
import matplotlib.pyplot as plt
#%matplotlib widget
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import pandas as pd

In [ ]:
# vector embeddings of each token
vec_embeddings = article_embeddings.numpy()

# color for each token for visuals
token_color = [np.random.rand(3,) for _ in range(num_articles)]

In [ ]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(vec_embeddings)
print(np.shape(pca_result))

In [ ]:
df = pd.DataFrame(pca_result, columns=['pca_0', 'pca_1'], index=article_names)
display(df)

In [ ]:
plt.figure(figsize=(16,8))
sns.scatterplot(
    x='pca_0', y='pca_1',
    data=df,
    legend='full'
)
plt.grid()

ax = plt.gca()
for (k, v), color in zip(df.iterrows(), token_color):
    ax.annotate(k, v, c=color)

#for idx, (article, color) in enumerate(zip(article_names, token_color)):
#    plt.text(-7 + idx*1, -10, article, fontsize='medium', color=color)

In [ ]:
tsne = TSNE(n_components=2, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(vec_embeddings)
print(np.shape(tsne_results))

In [ ]:
df = pd.DataFrame(tsne_results, columns=['tsne_0', 'tsne_1'], index=article_names)
display(df)

In [ ]:
plt.figure(figsize=(16,8))
sns.scatterplot(
    x="tsne_0", y="tsne_1",
    data=df,
    legend="full"
)
plt.grid()

ax = plt.gca()
for (k, v), color in zip(df.iterrows(), token_color):
    ax.annotate(k, v, c=color)

#for idx, (token, color) in enumerate(zip(tokens, token_color)):
#    plt.text(-400 + idx*50, -400, token, fontsize='medium', color=color)

# Dimensionality Reduction projection into 3D space

In [ ]:
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
pca = PCA(n_components=3)
pca_result = pca.fit_transform(vec_embeddings)
print(np.shape(pca_result))

In [ ]:
df = pd.DataFrame(pca_result, columns=['pca_0', 'pca_1', 'pca_2'], index=article_names)
display(df)

In [ ]:
fig = plt.figure()
ax = Axes3D(fig)

ax.scatter(df['pca_0'], df['pca_1'], df['pca_2'], c=token_color)
plt.grid()

In [ ]:
print(np.shape(vec_embeddings))

# Training Data

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Conv1D, MaxPooling1D, InputLayer, Input

In [ ]:
originality_scores = list(map(int, dataset.loc['Originality Score'].values))
print(f'Num Originality Scores = {len(originality_scores)}')
print(originality_scores)

In [ ]:
model = Sequential([
    Input(shape=(768,), name='input'),
    Dense(64, activation='relu', name='dense_0'),
    Dense(64, activation='relu', name='dense_1'),
    Dense(1000, name='dense_2'),
])
model.summary()

In [ ]:
'''
inputs = tf.keras.Input(shape=(768,), name='digits')
x = tf.keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = tf.keras.layers.Dense(64, activation='relu', name='dense_2')(x)
x = tf.keras.layers.Dense(1000, name='predictions')(x)
#outputs = tf.keras.layers.Softmax()(x)
outputs = x;


model = tf.keras.Model(inputs=inputs, outputs=outputs);
model.summary()
'''

In [ ]:
print(num_articles)
num_input_articles = 40
num_test_articles = num_articles - num_input_articles

In [ ]:
input_vectors = vec_embeddings[:num_input_articles]
print(np.shape(input_vectors))

x_train = np.asarray(input_vectors)
print(x_train.shape)

output_vectors = np.array([originality_scores[i] for i in range(num_input_articles)])
print(np.shape(output_vectors))

y_train = output_vectors

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),  # Optimizer
              # Loss function to minimize
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              # List of metrics to monitor
              metrics=['accuracy'])

print('# Fit model on training data')
history = model.fit(x_train, y_train,
                    batch_size=10,
                    epochs=1500,)
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    #validation_data=(x_val, y_val))

print('\nhistory dict:', history.history)

In [ ]:
x_test = vec_embeddings[num_input_articles:]
y_test = np.array([originality_scores[i] for i in range(num_input_articles, num_articles)])
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)